In [ ]:
%matplotlib inline

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from ipywidgets import interact

plt.rcParams['figure.figsize'] = [14, 10]

from matplotlib.colors import LinearSegmentedColormap

def getRandomColorMap(num_colors, bg_color=1):
    colors = np.random.rand(num_colors, 3) * 0.75
    colors[0, :] = bg_color
    colors = tuple(map(tuple, colors))

    labelColorMap = LinearSegmentedColormap.from_list('labelColorMap', colors, N=num_colors)

    return labelColorMap

def multiplot(lines, rows, images, cmap, title, save=False):
    plt.figure(figsize=(20,10))
    for i in np.arange(lines*rows):
        
        plt.subplot(lines, rows, i+1)
        plt.imshow(images[i], vmax=255, cmap=cmap[i])
        plt.title(title[i])
        plt.xticks([])
        plt.yticks([])
    
    if save:
        plt.savefig('img.png')
    plt.show()

Load the image and convert it in grayscale image.

In [ ]:
image = cv2.imread('Images/baboon.png')

rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

multiplot(1, 2, (rgb_image, gray_image), (cm.viridis, cm.gray), ('Original image', 'Gray image')) 

The most basic linear filtering is a moving average of the image.  

In [ ]:
def average_filter(kx, ky):
    kernel = np.ones((ky, kx)) / (kx*ky)
    avg_image = cv2.filter2D(gray_image, -1, kernel)
    
    multiplot(1, 2, (gray_image, avg_image), (cm.gray, cm.gray), ('Gray image', 'Average filtered image'))
    
interact(average_filter, kx=(1, 11, 2), ky=(1, 11, 2))

What is the difference between average filtering and uniform blurring?

In [ ]:
def blurring_filter(kx, ky):
    kernel = (ky, kx)
    blur_image = cv2.blur(gray_image, kernel)
    
    multiplot(1, 2, (gray_image, blur_image), (cm.gray, cm.gray), ('Gray image', 'Blurred image'))
    
interact(blurring_filter, kx=(1, 11, 2), ky=(1, 11, 2))

In [ ]:
avg_image = cv2.filter2D(gray_image, -1, np.ones((3, 3)) / (3*3))
blur_image = cv2.blur(gray_image, (3, 3))
box_image = cv2.boxFilter(gray_image, -1, (3, 3), normalize=True)

(avg_image.flatten() == blur_image.flatten()).sum() / avg_image.size, (avg_image.flatten() == box_image.flatten()).sum() / avg_image.size

What is the difference between uniform and gaussian blurring?

In [ ]:
def gaussian_filter(kx, ky):
    kernel = (ky, kx)
    gauss_image = cv2.GaussianBlur(gray_image, kernel, sigmaX=0)     
    
    multiplot(1, 2, (gray_image, gauss_image), (cm.gray, cm.gray), ('Gray image', 'Gaussian blurred image'))
    
interact(gaussian_filter, kx=(1, 11, 2), ky=(1, 11, 2))

We can obtain a high pass filtered image by applying a low pas filter and then taking the complementary.

In [ ]:
def hp_filter(kx, ky):
    kernel = (ky, kx)
    gauss_image = cv2.GaussianBlur(gray_image, kernel, 3)
    
    hp = gray_image - gauss_image
    
    multiplot(1, 2, (gray_image, hp), (cm.gray, cm.gray), ('Gray image', 'HP filter image'))
    
interact(hp_filter, kx=(1, 21, 2), ky=(1, 21, 2))

Compute and display the histogram of a grayscale image

In [ ]:
cameraman_gray = cv2.imread('Images/cameraman.jpg', cv2.IMREAD_GRAYSCALE)

plt.imshow(cameraman_gray, cmap=cm.gray)

In [ ]:
hist_np, bins = np.histogram(cameraman_gray.ravel(), 256, [0,256])

hist_cv = cv2.calcHist( [cameraman_gray], [0], None, [256], [0,256])

plt.hist(cameraman_gray.ravel(), bins=256, range=(0,255))
plt.show()

Basic thresholding

In [ ]:
cameraman_threshold = cameraman_gray > 100

plt.imshow(cameraman_threshold, cmap=cm.gray)
plt.show()

#### Thresholding with OpenCV
Determine the v1 and v2 values for the following threshold types:  
cv.THRESH_BINARY
$$\text{th_image}(x,y)=\left\{
                \begin{array}{ll}
                  \texttt{v1} & \text{if img$(x,y)$ > thresh}\\
                  \texttt{v2} & \text{otherwise}
                \end{array}
              \right.
$$

cv.THRESH_BINARY_INV
$$\text{th_image}(x,y)=\left\{
                \begin{array}{ll}
                  \texttt{v1} & \text{if img$(x,y)$ > thresh}\\
                  \texttt{v2} & \text{otherwise}
                \end{array}
              \right.
$$

cv.THRESH_TRUNC
$$\text{th_image}(x,y)=\left\{
                \begin{array}{ll}
                  \texttt{v1} & \text{if img$(x,y)$ > thresh}\\
                  \texttt{v2} & \text{otherwise}
                \end{array}
              \right.
$$

cv.THRESH_TO_ZERO
$$\text{th_image}(x,y)=\left\{
                \begin{array}{ll}
                  \texttt{v1} & \text{if img$(x,y)$ > thresh}\\
                  \texttt{v2} & \text{otherwise}
                \end{array}
              \right.
$$

cv.THRESH_TO_ZERO_INV
$$\text{th_image}(x,y)=\left\{
                \begin{array}{ll}
                  \texttt{v1} & \text{if img$(x,y)$ > thresh}\\
                  \texttt{v2} & \text{otherwise}
                \end{array}
              \right.
$$

In [ ]:
grad = cv2.imread('Images/gradient.png', cv2.IMREAD_GRAYSCALE)

value = 150

ret, thresh1 = cv2.threshold(grad, value, 255, cv2.THRESH_BINARY)
ret, thresh2 = cv2.threshold(grad, value, 255, cv2.THRESH_BINARY_INV)
ret, thresh3 = cv2.threshold(grad, value, 255, cv2.THRESH_TRUNC)
ret, thresh4 = cv2.threshold(grad, value, 255, cv2.THRESH_TOZERO)
ret, thresh5 = cv2.threshold(grad, value, 255, cv2.THRESH_TOZERO_INV)

multiplot(2, 3,
          (grad, thresh1, thresh2, thresh3, thresh4, thresh5),
          (cm.gray, cm.gray, cm.gray, cm.gray, cm.gray, cm.gray),
          ('Original Image', 'THRESH_BINARY', 'THRESH_BINARY_INV',
           'THRESH_TRUNC', 'THRESH_TOZERO', 'THRESH_TOZERO_INV'))

In [ ]:
value = 127

ret, thresh1 = cv2.threshold(cameraman_gray, value, 255, cv2.THRESH_BINARY)
ret, thresh2 = cv2.threshold(cameraman_gray, value, 255, cv2.THRESH_BINARY_INV)
ret, thresh3 = cv2.threshold(cameraman_gray, value, 255, cv2.THRESH_TRUNC)
ret, thresh4 = cv2.threshold(cameraman_gray, value, 255, cv2.THRESH_TOZERO)
ret, thresh5 = cv2.threshold(cameraman_gray, value, 255, cv2.THRESH_TOZERO_INV)

multiplot(2, 3,
         (cameraman_gray, thresh1, thresh2, thresh3, thresh4, thresh5),
         (cm.gray, cm.gray, cm.gray, cm.gray, cm.gray, cm.gray),
         ('Original Image', 'THRESH_BINARY', 'THRESH_BINARY_INV',
          'THRESH_TRUNC', 'THRESH_TOZERO', 'THRESH_TOZERO_INV'))

In the previous examples, we had to chose the threshold value. We can use Otsu's algorithm to determine it.

In [ ]:
rice = cv2.imread('Images/rice.jpg', cv2.IMREAD_GRAYSCALE)

plt.hist(rice.ravel(), bins=256, range=(0,255))
plt.show()

ret, thresh1 = cv2.threshold(rice, 127, 255, cv2.THRESH_BINARY)

ret, otsu = cv2.threshold(rice, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

multiplot(1, 3,
         (rice, thresh1, otsu),
         (cm.gray, cm.gray, cm.gray),
         ('Original Image', 'THRESH_BINARY', 'THRESH_BINARY + OTSU'))

You can fine-tune the result by applying this algorithm on the different parts of the image.

In [ ]:
n = 5
h = np.linspace(0, rice.shape[0], n+1).astype('int')
out = []
for i in range(n):
    ret, th = cv2.threshold(rice[h[i]: h[i+1], :], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    out.append(th)

out = np.concatenate(out, axis=0)

multiplot(1, 3,
         (rice, otsu, out),
         (cm.gray, cm.gray, cm.gray),
         ('Original Image', 'OTSU', 'OTSU BY PARTS'))

In [ ]:
def otsu_parts(img, n, direction):
    
    out = []
    
    if direction == 'horizontal':
        l = img.shape[0]
        h = np.linspace(0, l, n+1).astype('int')
        
        for i in range(n):
            ret, th = cv2.threshold(img[h[i]: h[i+1], :], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            out.append(th)
    else:
        l = img.shape[1]
        h = np.linspace(0, l, n+1).astype('int')
        
        for i in range(n):
            ret, th = cv2.threshold(img[:, h[i]: h[i+1]], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            out.append(th)
    
    
    return np.concatenate(out, axis=0)

light_gradient = np.arange(start=132, stop=-133, step=-1) / 3
non_uniform_lightning = (np.reshape(light_gradient, (265, 1))) * np.ones((1, 250), dtype=int)

img_nu = np.clip(rice+non_uniform_lightning, 0, 255).astype(np.uint8)

plt.hist(img_nu.ravel(), bins=256, range=(0,255))

ret, otsu = cv2.threshold(img_nu, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
otsu_p = otsu_parts(img_nu, 5, 'horizontal')

multiplot(2, 2,
         (rice, non_uniform_lightning, otsu, otsu_p),
         (cm.gray, cm.gray, cm.gray, cm.gray),
         ('Original Image', 'Non-uniform lightning', 'OTSU', 'OTSU BY PARTS'))

Gaussian noise

In [ ]:
pattern = cv2.imread('Images/pattern_inspection2.png', cv2.IMREAD_GRAYSCALE)

gaussian_noise = np.zeros_like(pattern, dtype=float)
mean = 0.
std = 10.
cv2.randn(gaussian_noise, mean, std)

pattern_noisy = np.clip(pattern+gaussian_noise, 0, 255).astype(np.uint8)

multiplot(1, 2,
         (pattern, pattern_noisy),
         (cm.gray, cm.gray),
         ('Original image', 'Gaussian noise'))

Salt and pepper noise

In [ ]:
salt_pepper_noise = np.zeros_like(cameraman_gray)
cv2.randu(salt_pepper_noise, 0, 255)
cameraman_noisy = np.where(salt_pepper_noise < 10, 0, np.where(salt_pepper_noise > 240, 255, cameraman_gray))

multiplot(1, 2,
         (cameraman_gray, cameraman_noisy),
         (cm.gray, cm.gray),
         ('Original image', 'Salt and pepper noise'))

In [ ]:
cameraman_noisy_uniform_blur = cv2.blur(cameraman_noisy, (7, 7))
cameraman_noisy_gaussian_blur = cv2.GaussianBlur(cameraman_noisy, (7, 7), 0)
cameraman_noisy_bilateral = cv2.bilateralFilter(cameraman_noisy, -1, 25, 11)

multiplot(2, 2,
         (cameraman_gray, cameraman_noisy_uniform_blur, cameraman_noisy_gaussian_blur, cameraman_noisy_bilateral),
         (cm.gray, cm.gray, cm.gray, cm.gray),
         ('Original image', 'Uniform blur', 'Gaussian blur', 'Bilateral filtering'))

In [ ]:
salt_pepper_noise = np.zeros_like(cameraman_gray)
cv2.randu(salt_pepper_noise, 0, 255)
cameraman_noisy = np.where(salt_pepper_noise < 30, 0, np.where(salt_pepper_noise > 200, 255, cameraman_gray))

cameraman_noisy_median = cv2.medianBlur(cameraman_noisy, 5)

multiplot(1, 3,
         (cameraman_gray, cameraman_noisy, cameraman_noisy_median),
         (cm.gray, cm.gray, cm.gray),
         ('Salt and pepper noise', 'Noisy image', 'Median blur'))

In [ ]:
def non_uniform_lightning_like(img, weight):
    width = img.shape[1]
    height = img.shape[0]
    
    steps_y = np.arange( start=0.0, stop=1.0, step=1.0/height)
    light_gradient_y = np.cos( ( 2.0 * ( steps_y * steps_y - steps_y) + 1.0)* np.pi)[:,np.newaxis]

    steps_x = np.arange( start=0.0, stop=1.0, step=1.0/width)
    light_gradient_x = np.cos( steps_x * np.pi)[np.newaxis,:]

    return ( weight * light_gradient_y * light_gradient_x)

nul = non_uniform_lightning_like(rice, 50)
rice_nul = np.clip(rice + nul, 0, 255).astype(np.uint8)

multiplot(1, 3,
         (rice, nul, rice_nul),
         (cm.gray, cm.gray, cm.gray),
         ('Original Image', 'Non Uniform Lightning', 'Image + Non Uniform Lightning'))

In [ ]:
k = 19
kernel = np.ones((k, k))

erd = cv2.erode(rice_nul, kernel, iterations=1)
dlt = cv2.dilate(rice_nul, kernel, iterations=1)

multiplot(1, 2,
         (erd, dlt),
         (cm.gray, cm.gray),
         ('Erosion', 'Dilation'))

In [ ]:
gaussian_erd = cv2.GaussianBlur(erd, (k, k), 0)
gaussian_dlt = cv2.GaussianBlur(dlt, (k, k), 0)

multiplot(1, 2,
         (gaussian_erd, gaussian_dlt),
         (cm.gray, cm.gray),
         ('Background', 'Foreground'))

In [ ]:
def local_normalize_image(img, k):
    kernel = np.ones((k, k))
    
    erd = cv2.erode(img, kernel, iterations=1)
    dlt = cv2.dilate(img, kernel, iterations=1)
    
    gaussian_erd = cv2.GaussianBlur(erd, (k, k), 0)
    gaussian_dlt = cv2.GaussianBlur(dlt, (k, k), 0)
    
    norm_img = (img - gaussian_erd)/(gaussian_dlt - gaussian_erd + 1) * 255
    
    return np.clip(norm_img, 0, 255).astype(np.uint8)

rice_nul_norm = local_normalize_image(rice_nul, 19)

ret, thresh1 = cv2.threshold(rice_nul, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
ret, thresh2 = cv2.threshold(rice_nul_norm, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

multiplot(2, 2,
         (rice_nul, rice_nul_norm, thresh1, thresh2),
         (cm.gray, cm.gray, cm.gray, cm.gray),
         ('Image + Non Uniform Lightning', 'Uniformized Image', 'OTSU', 'OTSU + uniformized'))